In [1]:
from redisvl.extensions.router.routes import Route, RoutingConfig

In [2]:
# Define individual routes manually with metadata
politics = Route(
    name="politics",
    references=[
        "isn't politics the best thing ever",
        "why don't you tell me about your political opinions"
    ],
    metadata={"priority": 1}
)

chitchat = Route(
    name="chitchat",
    references=[
        "hello",
        "how's the weather today?",
        "how are things going?"
    ],
    metadata={"priority": 2}
)

In [3]:
routes = [politics, chitchat]

In [4]:
config = RoutingConfig(max_k=2, distance_threshold=1.0, accumulation_method="avg")

In [5]:
from redisvl.extensions.router.semantic import SemanticRouter
import redis

# Create SemanticRouter named "topic-router"
redis_client = redis.Redis()
routes = [politics, chitchat]
topic_router = SemanticRouter(
    name="topic-router",
    routes=routes,
    routing_config=config,
    redis_client=redis_client,
    overwrite=True
)

23:37:26 redisvl.index.index INFO   Index already exists, overwriting.


In [7]:
assert topic_router.routes == routes
assert topic_router.name == "topic-router"
assert topic_router.name == topic_router._index.name == topic_router._index.prefix
assert topic_router.routing_config == config

In [8]:
# Query topic-router with behavior based on the config
topic_router("don't you love politics?")

[{'name': 'politics',
  'references': ["isn't politics the best thing ever",
   "why don't you tell me about your political opinions"],
  'metadata': {'priority': '1'},
  'score': 0.3825837373735},
 {'name': 'chitchat',
  'references': ['hello', "how's the weather today?", 'how are things going?'],
  'metadata': {'priority': '2'},
  'score': 0.8872345884643332}]

In [9]:
topic_router("how are you")

[{'name': 'chitchat',
  'references': ['hello', "how's the weather today?", 'how are things going?'],
  'metadata': {'priority': '2'},
  'score': 0.5357088247936667},
 {'name': 'politics',
  'references': ["isn't politics the best thing ever",
   "why don't you tell me about your political opinions"],
  'metadata': {'priority': '1'},
  'score': 0.8782881200315}]

Need to work on a way to properly handle the "sum" strategy... Adding route references by vector distance when 0 is "more similar" means we would have to flip polarity in order to capture the right signal.

Avg is a safer strategy in general.. but assumes that the variance in the amount of references per route is not too high.

In [13]:
config = RoutingConfig(distance_threshold=0.1, accumulation_method="avg")

topic_router.update_routing_config(config)

In [14]:
# distance threshold is too low
topic_router("hello world")

[]

In [15]:
topic_router("hello world", distance_threshold=0.3)

[{'name': 'chitchat',
  'references': ['hello', "how's the weather today?", 'how are things going?'],
  'metadata': {'priority': '2'},
  'score': 0.243986725807}]